In [13]:
import xarray as xr
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn

In [2]:
WINDOW_IN_MIN = 180   # 3 hours window for input
WINDOW_OUT_MIN = 60   # 1 hour ahead for output

flare_order = {"A": 0, "B": 1, "C": 2, "M": 3, "X": 4}

def get_highest_class(classes):
    if len(classes) == 0:
        return "A"
    return max(classes, key=lambda c: flare_order[c])

def make_future_labels(index, flare_df, horizon_min=60):

    # DatetimeIndex of the 1-min irradiance series
    # flare_df: dataframe with ['flare_time', 'class_letter']

    fl_times = flare_df['flare_time'].values
    fl_classes = flare_df['class_letter'].values
    n_events = len(fl_times)

    labels = []
    j = 0

    for t in index.values:
        # advance pointer past any flares before t
        while j < n_events and fl_times[j] < t:
            j += 1

        h_end = t + np.timedelta64(horizon_min, 'm')
        k = j
        classes = []

        while k < n_events and fl_times[k] <= h_end:
            classes.append(fl_classes[k])
            k += 1

        labels.append(get_highest_class(classes))

    return np.array(labels)

def build_windows(irrad_1m, labels, window_in_min=180):
    data = irrad_1m[['short_xray', 'long_xray']].values
    X_list, y_list = [], []

    for i in range(window_in_min, len(irrad_1m)):
        x_window = data[i-window_in_min:i, :]
        X_list.append(x_window)
        y_list.append(labels[i])

    X = np.stack(X_list)
    y = np.array(y_list)
    return X, y


In [3]:
def process_year(year, base_irrad_dir="", base_flsum_dir=""):
    print(f"\n=== Processing year {year} ===")
    irrad_path = f"{base_irrad_dir}irrad_{year}.nc"
    flsum_path = f"{base_flsum_dir}flsum_{year}.nc"

    # 1) Load irrad
    irrad_ds = xr.open_dataset(irrad_path)
    irrad_df = irrad_ds[['a_flux', 'b_flux']].to_dataframe().reset_index()

    irrad_df.rename(columns={
        'time': 'timestamp',
        'a_flux': 'short_xray',
        'b_flux': 'long_xray'
    }, inplace=True)

    irrad_df['timestamp'] = pd.to_datetime(irrad_df['timestamp'])
    irrad_df.set_index('timestamp', inplace=True)

    # 2) Resample to 1-min
    irrad_1m = irrad_df.resample('1min').mean().dropna()
    print("1-min irradiance shape:", irrad_1m.shape)

    # 3) Load flare summary
    flsum_ds = xr.open_dataset(flsum_path)
    flsum_df = flsum_ds[['flare_class']].to_dataframe().reset_index()

    flsum_df.rename(columns={'time': 'flare_time'}, inplace=True)
    flsum_df['flare_time'] = pd.to_datetime(flsum_df['flare_time'])
    flsum_df['class_letter'] = flsum_df['flare_class'].astype(str).str[0]

    valid_classes = ['A', 'B', 'C', 'M', 'X']
    flsum_df = flsum_df[flsum_df['class_letter'].isin(valid_classes)].copy()
    flsum_df.sort_values('flare_time', inplace=True)
    flsum_df.reset_index(drop=True, inplace=True)

    print("Number of flare events:", len(flsum_df))

    # 4) Future labels (next 1 hour)
    labels = make_future_labels(irrad_1m.index, flsum_df, horizon_min=WINDOW_OUT_MIN)
    print("Labels shape:", labels.shape)

    # 5) Build sliding windows
    X_year, y_year = build_windows(irrad_1m, labels, window_in_min=WINDOW_IN_MIN)
    print("Year", year, "X shape:", X_year.shape, "y shape:", y_year.shape)

    return X_year, y_year

In [20]:
base_irrad_dir = "C:/Users/ruthw/Desktop/irrad/"
base_flsum_dir = "C:/Users/ruthw/Desktop/flsum/"

In [21]:
YEARS = [2016, 2017, 2018]  # adjust to what you have

X_list = []
y_list = []

for year in YEARS:
    X_y, y_y = process_year(year, base_irrad_dir, base_flsum_dir)
    X_list.append(X_y)
    y_list.append(y_y)

X_all = np.concatenate(X_list, axis=0)
y_all = np.concatenate(y_list, axis=0)

print("Combined X:", X_all.shape)
print("Combined y:", y_all.shape)
print(pd.Series(y_all).value_counts())


=== Processing year 2016 ===
1-min irradiance shape: (501754, 2)
Number of flare events: 2084
Labels shape: (501754,)
Year 2016 X shape: (501574, 180, 2) y shape: (501574,)

=== Processing year 2017 ===
1-min irradiance shape: (497854, 2)
Number of flare events: 1541
Labels shape: (497854,)
Year 2017 X shape: (497674, 180, 2) y shape: (497674,)

=== Processing year 2018 ===
1-min irradiance shape: (499698, 2)
Number of flare events: 409
Labels shape: (499698,)
Year 2018 X shape: (499518, 180, 2) y shape: (499518,)
Combined X: (1498766, 180, 2)
Combined y: (1498766,)
A    1302308
B     148551
C      43148
M       4393
X        366
Name: count, dtype: int64


In [22]:
# Clean
X_all[~np.isfinite(X_all)] = 0.0
X_all = np.clip(X_all, 0.0, None)

# Log-transform
X_all = np.log10(X_all + 1e-10)

# Optional: global standardization
mean = X_all.mean(axis=(0, 1), keepdims=True)
std  = X_all.std(axis=(0, 1), keepdims=True) + 1e-6
X_all = (X_all - mean) / std

print("After transform, any NaN?", np.isnan(X_all).any(), " any Inf?", np.isinf(X_all).any())

After transform, any NaN? False  any Inf? False


In [23]:
np.save("X_multi.npy", X_all)
np.save("y_multi.npy", y_all)

In [25]:
X = np.load("X_multi.npy")
y = np.load("y_multi.npy")

# Manual encoding to keep class order fixed
label_map = {'A': 0, 'B': 1, 'C': 2, 'M': 3, 'X': 4}
y_encoded = np.array([label_map[c] for c in y])
inv_label_map = {v: k for k, v in label_map.items()}

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.30, random_state=42, stratify=y_encoded
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

classes = np.array([0,1,2,3,4])
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float32)
print("Class weights:", class_weights)

Class weights: tensor([2.3017e-01, 2.0178e+00, 6.9470e+00, 6.8236e+01, 8.1964e+02])


# MODELS

In [26]:
print("X:", X.shape)
print("y:", y.shape)
pd.Series(y).value_counts()

X: (1498766, 180, 2)
y: (1498766,)


A    1302308
B     148551
C      43148
M       4393
X        366
Name: count, dtype: int64

In [27]:
label_map = {'A': 0, 'B': 1, 'C': 2, 'M': 3, 'X': 4}
inv_label_map = {v: k for k, v in label_map.items()}

y_encoded = np.array([label_map[c] for c in y])

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.30, random_state=42, stratify=y_encoded
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

Train: (1049136, 180, 2) Val: (224815, 180, 2) Test: (224815, 180, 2)


In [29]:
classes = np.array([0,1,2,3,4])

raw_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)

# Tame extreme imbalance
scaled_weights = np.sqrt(raw_weights)
scaled_weights = scaled_weights / scaled_weights.min()

class_weights = torch.tensor(scaled_weights, dtype=torch.float32)

print("Raw:", raw_weights)
print("Scaled:", scaled_weights)

Raw: [2.30170851e-01 2.01784086e+00 6.94700040e+00 6.82364878e+01
 8.19637500e+02]
Scaled: [ 1.          2.96086288  5.49380829 17.21801854 59.67408226]


In [30]:
class GOESDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # (180, 2) → (2, 180)
        x = self.X[idx].transpose(1, 0)
        return x, self.y[idx]

train_loader = DataLoader(GOESDataset(X_train, y_train), batch_size=256, shuffle=True)
val_loader   = DataLoader(GOESDataset(X_val, y_val), batch_size=512, shuffle=False)
test_loader  = DataLoader(GOESDataset(X_test, y_test), batch_size=512, shuffle=False)

In [31]:
# CNN

In [32]:
class CNN1D(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(2, 32, kernel_size=5, padding=2)
        self.bn1   = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2   = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn3   = nn.BatchNorm1d(128)

        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.global_pool(x).squeeze(-1)
        x = self.dropout(x)
        return self.fc(x)

In [33]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=1.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, targets):
        ce = F.cross_entropy(logits, targets, weight=self.alpha, reduction='none')
        pt = torch.exp(-ce)
        loss = ((1 - pt) ** self.gamma) * ce
        return loss.mean()

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN1D(n_classes=5).to(device)
criterion = FocalLoss(alpha=class_weights.to(device), gamma=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss, correct, total = 0, 0, 0

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            out = model(Xb)
            loss = criterion(out, yb)
            if train:
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * len(yb)
        preds = out.argmax(1)
        correct += (preds == yb).sum().item()
        total += len(yb)

    return total_loss / total, correct / total

In [35]:
EPOCHS = 10
for ep in range(1, EPOCHS+1):
    train_loss, train_acc = run_epoch(train_loader, train=True)
    val_loss, val_acc     = run_epoch(val_loader, train=False)

    print(f"Epoch {ep:02d} | "
          f"Train loss {train_loss:.4f}, acc {train_acc:.3f} | "
          f"Val loss {val_loss:.4f}, acc {val_acc:.3f}")

Epoch 01 | Train loss 0.7769, acc 0.776 | Val loss 0.7422, acc 0.792
Epoch 02 | Train loss 0.7384, acc 0.793 | Val loss 0.7695, acc 0.683
Epoch 03 | Train loss 0.7164, acc 0.797 | Val loss 0.7041, acc 0.766
Epoch 04 | Train loss 0.6959, acc 0.798 | Val loss 0.8490, acc 0.674
Epoch 05 | Train loss 0.6756, acc 0.801 | Val loss 0.6644, acc 0.802
Epoch 06 | Train loss 0.6544, acc 0.804 | Val loss 0.7340, acc 0.642
Epoch 07 | Train loss 0.6320, acc 0.806 | Val loss 0.6219, acc 0.797
Epoch 08 | Train loss 0.6128, acc 0.809 | Val loss 0.6053, acc 0.800
Epoch 09 | Train loss 0.5973, acc 0.811 | Val loss 0.6176, acc 0.759
Epoch 10 | Train loss 0.5773, acc 0.815 | Val loss 0.6441, acc 0.749


In [36]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        out = model(Xb)
        preds = out.argmax(1)
        all_preds.append(preds.cpu().numpy())
        all_true.append(yb.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_true  = np.concatenate(all_true)

print(classification_report(all_true, all_preds,
                            target_names=['A','B','C','M','X']))

print(confusion_matrix(all_true, all_preds))

              precision    recall  f1-score   support

           A       0.95      0.77      0.85    195347
           B       0.27      0.67      0.38     22282
           C       0.36      0.60      0.45      6472
           M       0.29      0.60      0.39       659
           X       0.47      0.85      0.61        55

    accuracy                           0.75    224815
   macro avg       0.47      0.70      0.54    224815
weighted avg       0.87      0.75      0.79    224815

[[149524  39458   5696    633     36]
 [  6382  14884   1008      8      0]
 [   633   1586   3890    347     16]
 [    33     66    161    398      1]
 [     0      0      2      6     47]]


In [37]:
from sklearn.metrics import classification_report, confusion_matrix

# Ground truth: ≥C
y_true_C = (all_true >= 2).astype(int)
y_pred_C = (all_preds >= 2).astype(int)

print("Binary ≥C classification report:")
print(classification_report(
    y_true_C,
    y_pred_C,
    target_names=["<C (A/B)", "≥C (C/M/X)"]
))

print("Confusion matrix (≥C):")
print(confusion_matrix(y_true_C, y_pred_C))

Binary ≥C classification report:
              precision    recall  f1-score   support

    <C (A/B)       0.99      0.97      0.98    217629
  ≥C (C/M/X)       0.40      0.68      0.50      7186

    accuracy                           0.96    224815
   macro avg       0.69      0.82      0.74    224815
weighted avg       0.97      0.96      0.96    224815

Confusion matrix (≥C):
[[210248   7381]
 [  2318   4868]]


In [38]:
# Ground truth: ≥M
y_true_M = (all_true >= 3).astype(int)
y_pred_M = (all_preds >= 3).astype(int)

print("Binary ≥M classification report:")
print(classification_report(
    y_true_M,
    y_pred_M,
    target_names=["<M (A/B/C)", "≥M (M/X)"]
))

print("Confusion matrix (≥M):")
print(confusion_matrix(y_true_M, y_pred_M))

Binary ≥M classification report:
              precision    recall  f1-score   support

  <M (A/B/C)       1.00      1.00      1.00    224101
    ≥M (M/X)       0.30      0.63      0.41       714

    accuracy                           0.99    224815
   macro avg       0.65      0.81      0.70    224815
weighted avg       1.00      0.99      1.00    224815

Confusion matrix (≥M):
[[223061   1040]
 [   262    452]]
